<a href="https://colab.research.google.com/github/donguk99/TSP/blob/master/TSP_TW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code start

In [7]:
import numpy as np
import cvxpy as cp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# Input parameters

In [8]:
# Graph
"""
m: number of salesman
d: distance graph
s: service time for each node
tw: time windows
"""
#=======================input======================
m=1
d = [[0,  2,  3,  4],
     [2,  0,  3,  2],
     [3,  3,  0,  5],
     [4,  2,  5,  0]]
d=np.array(d)
n=d.shape[0]

# service time
s = np.zeros((n,1))
s[0]=0

# time window
tw = np.array([[0, 1000],
               [5,10],
               [0,30],
               [3,10]])

#======================input end=======================


e = tw[:, 0]
l = tw[:, 1]
M = 100000


# Variables

In [9]:
a = cp.Variable((n,1))
p = cp.Variable((n,1))
x = cp.Variable((n,n), boolean=True)

# Objective, Constraints

In [10]:
#objective
objective = cp.Minimize(cp.trace(d.T@x))

#constraints
ones= np.ones((n,1))
constraints = []
constraints += [x[0,:]@ones == m] # m명이 출발노드에서 출발  # m==1이면 TSP problem
constraints += [x[:,0]@ones == m] # m명이 출발노드로 도착
constraints += [x[1:,:]@ones == 1] # 각 노드는 1명에 의해 한번만 방문 
constraints += [x[:,1:].T@ones == 1]
constraints += [cp.diag(x) == 0]

# TW constraints
constraints += [p[0] == 0]
for i in range(n):
  constraints += [e[i] <= p[i]]
  constraints += [p[i] <= l[i]]
  if i != 0:
    constraints += [a[i] <= p[i]-s[i]]

for i in range(n):
  for j in range(n):
    if i != j:
      constraints += [a[j] >= p[i]+d[i,j]-(1-x[i,j])*M]
      constraints += [a[j] <= p[i]+d[i,j]+(1-x[i,j])*M]

# subtour 찾는 함수

In [11]:
def get_subtours(routes,m)->list:
  # covert routes to subtours
  # example) [(0,1), (1,2), (2,0), (3,4), (4, 3)] -> [(0,1,2), (3,4)]
  # return subtours

  subtours = []
  r=routes.copy()

  # depart
  depart = r[:m]
  r= r[m:]

  # route (salesman)
  for route in depart:
    route=list(route)
    while r and route[-1] != 0:
      for i, j in enumerate(r):
        if route[-1] == j[0]:
            route.append(j[1])
            del (r[i])
            break
    if route[-1] == 0:
      route.pop()
    subtours.append(route)

  # route (left)
  while len(r) != 0:
    route = list(r.pop())
    l = 0
    while len(route) > l:
      l = len(route)
      for i, j in enumerate(r):
        if route[-1] == j[0]:
          if j[1] != route[0]:
            route.append(j[1])
            del (r[i])
          else:
            del (r[i])
    subtours.append(route)

  return subtours

# prob solve

In [12]:
# mTSP subtour elimination
iter = 0
subtours=[]
while len(subtours)!=m:
  prob = cp.Problem(objective, constraints)
  # prob.solve(verbose="True", eps=1e-6)
  prob.solve()

  # find subtours
  routes=[(i,j) for i in range(n) for j in range(n) if x[i,j].value==1]
  subtours = get_subtours(routes,m)

  print(f'iteration:{iter}')
  print(f'number of constraints:{len(constraints)}')
  print(f'number of subtours:{len(subtours)}')
  print(f'cost:{prob.value}')
  print(f'subtour:{subtours} \n')

  S=[]
  for subtour in subtours:
    V=[0 for _ in range(n)]
    for i in subtour:
      V[i] = 1
    V=np.array(V)
    S.append(V)

  # add constraints
  for V in S:
    if m==1:
      constraints += [V@(x@V.T) <= sum(V)-1]
    elif V[0] != 1:
      constraints += [V@(x@V.T) <= sum(V)-1]
  iter+=1  

iteration:0
number of constraints:41
number of subtours:1
cost:12.0
subtour:[[0, 2, 1, 3]] 



# 확인

In [13]:
a = a.value
p = p.value
x = x.value
TorF = []
for i in range(n):
  for j in range(n):
    if i != j:
      TorF.append(a[j] <= p[i]+d[i,j]+(1-x[i,j])*M)
      TorF.append(a[j] >= p[i]+d[i,j]-(1-x[i,j])*M)

TorF

[array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True])]

In [14]:
a,p

(array([[14.],
        [ 6.],
        [ 3.],
        [ 8.]]), array([[-0.],
        [ 6.],
        [ 3.],
        [10.]]))

In [15]:
x

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])